# Core (Facts) Models

In [ ]:
import os
from pathlib import Path

from dotenv import load_dotenv
from sqlalchemy import create_engine
from snowflake.sqlalchemy import URL

In [ ]:
PROJ_ROOT = Path().resolve().parents[3]
env_file_dir = PROJ_ROOT / '.env'
_ = load_dotenv(env_file_dir, verbose=True)

## About

Develop queries for **mart (core)** models using DBT's `intermediate` models.

The version of these queries that does not use `intermediate` models is found in `from-staging/11_00_core_mart_queries.ipynb`.

### Order Facts (`fct_orders`)

#### Objective

Develop a data model to facilitate querying aggregated order performance.

#### Constraints

By definition, since this is a core model, the result should be sufficiently granular to be used by multiple business units.

#### Business Questions

1. Get our 20 biggest orders by dollar value and order size, per state.
2. Consider the promo discount percentage to be: 100 X discount/order cost. Get the orders which had the 50 biggest discounts (by percentage) applied to them. Which states did these top 50 orders come from?
3. Prepare a daily summary showing order metrics
   - total order value
   - total number of orders
   - average order quantity
4. What is our peak time of the day (morning, afternoon, evening, etc.) for receiving orders?
5. How close are our estimated and actual delivery times on average, per state? Which 10 states have the most delayed deliveries?
6. How many orders are currently being prepared for delivery?

### Notes

1. This notebook supports <kbd>Run</kbd> > <kbd>Run All Cells</kbd>.

## User Inputs

In [ ]:
#

In [ ]:
engine = create_engine(
    URL(
        drivername="driver",
        account=os.getenv("UPLIMIT_SNOWFLAKE_ACCOUNT"),
        user=os.getenv("UPLIMIT_SNOWFLAKE_USER"),
        password=os.getenv("UPLIMIT_SNOWFLAKE_PASS"),
        warehouse=os.getenv("UPLIMIT_SNOWFLAKE_WAREHOUSE"),
        role=os.getenv("UPLIMIT_SNOWFLAKE_ROLE"),
        database=os.getenv("UPLIMIT_SNOWFLAKE_DB_NAME"),
        schema=os.getenv("UPLIMIT_SNOWFLAKE_SCHEMA"),
    )
)

## Connect

Load Jupyter SQL extension

In [ ]:
%load_ext sql

Connect to DuckDB database

In [ ]:
%sql engine --alias connection

## Models

### `marts/core/fct_orders`

In [ ]:
%%sql
WITH order_summary AS (
    SELECT order_id,
           created_at,
           state_name,
           order_cost,
           promo_id,
           discount,
           order_total,
           total_order_size,
           num_unique_products,
           estimated_delivery_at,
           delivered_at,
           status
    FROM int_orders_joined_to_addresses_promos
),
orders_with_delivery_details AS (
    SELECT *,
           datediff(
               second, created_at, estimated_delivery_at
           ) AS estimated_delivery_time_seconds,
           datediff(second, created_at, delivered_at) AS delivery_time_seconds,
           (
               CASE
                   WHEN delivered_at > estimated_delivery_at
                   THEN ABS(
                       DATEDIFF(second, delivered_at, estimated_delivery_at)
                   )
                   ELSE NULL
               END
           ) AS delivery_delay_seconds
    FROM order_summary
)
SELECT *
FROM orders_with_delivery_details

## Disconnect

Close connection

In [ ]:
%sql --close connection